In [1]:
import csoundengine as ce

end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance


In [29]:
from csoundengine import OfflineEngine
e = OfflineEngine()




--Csound version 6.19 (double samples) May  8 2024
[commit: 937aa495e250d758efaa28e9697a8b75237c80cd]
libsndfile-1.2.0
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
writing 1024-byte blks of shorts to /tmp/csoundengine-7fprcsdv.wav (WAV)
SECTION 1:


In [30]:
e.compile(r'''
instr pitchtrack
  itabnum = p4   ; the sound source
  itabfreq = p5  ; the table where to put tracked pitch. The table must be big enough
  itabsize = ftlen(itabfreq)
  ifftsize = 2048
  ifftsize2 = ifftsize * 2  ; this second anaylsis helps smooth the result
  iwtype = 0        ; hamming
  kcount init 1     ; the first element is the element count
  aouts[] loscilx 1, 1, itabnum, 4, 1
  a0 = aouts[0]     ; only analyze first channel
  a0d = delay(a0, ifftsize2 * 0.5 / sr)  ; shift the original signal
  fsig  pvsanal a0d, ifftsize, 512, ifftsize, iwtype
  fsig2 pvsanal a0, ifftsize2, 512, ifftsize2, iwtype
  kfreq, kamp pvspitch fsig, 0.05
  kfreq2, kamp2 pvspitch fsig2, 0.07
  if kcount >= itabsize || detectsilence(a0, db(-90), 0.1) == 1 then
    turnoff
  endif
  if (kfreq2 == 0 && kfreq == 0 ? 1 : kfreq / kfreq2) < 0.5 then
    kfreq = kfreq
  endif
  tabw kfreq, kcount, itabfreq
  kcount += 1
  tabw kcount, 0, itabfreq
  ; To validate the analysis, we output the original sound and the resynthesized audio
  outs a0d, buzz(kfreq > 0 ? kamp : 0, kfreq, 5, -1)
endin
''')

In [31]:
# sndtab = e.readSoundfile("/home/em/Lib/snd/samples/test/piano-pitchtest-2.flac")
sndtab = e.readSoundfile("/home/em/Lib/snd/samples/speech/colours-german-male-1.flac")

sndtabinfo = e.tableInfo(sndtab)

In [32]:
duration = sndtabinfo.duration
numcycles = int(duration * e.sr / e.ksmps)
freqtab = e.makeEmptyTable(numcycles + 100)
e.sched('pitchtrack', 0, duration + 0.1, sndtab, freqtab)

'pitchtrack'

In [33]:
e.perform()
# Now retrieve information
outarr = e.getTableData(freqtab)
datalen = int(outarr[0])
print(f"{datalen=}")
freqs = outarr[1:1+datalen]
# Finished with the engine, stop it so that we can access the generated soundfile
# for validation. This is just to show that the output file is only fully formed
# after the engine is stopped
renderjob = e.stop()
# Open the outfile in the default application
# renderjob.openOutfile()


datalen=7472


end of score.		   overall amps:  0.89328  0.66084
	   overall samples out of range:        0        0
0 errors in performance
512 1024 sample blks of shorts written to /tmp/csoundengine-7fprcsdv.wav (WAV)


In [34]:
renderjob.openOutfile(app='mhwaveedit')

mhWaveEdit: playing file as 48000 Hz, float, Stereo


In [9]:
!ll /usr/bin/mhwave*

.rwxr-xr-x 536k root  1 Apr 08:57 /usr/bin/mhwaveedit*
